This notebook was made using code from Avery Bates to reformat images from padchest and mimic for use in the medical imaging pipeline. This is left more for example then for use. 

In [ ]:
import os,sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import shutil
import numpy as np
import pandas as pd
import pathlib
import torch
import torchvision
import torchxrayvision as xrv
import ast
import math
import re
import skimage
from PIL import Image
from torchvision.transforms.functional import to_pil_image

from concurrent.futures import ThreadPoolExecutor
import os
import skimage.io
import skimage.color

In [ ]:
#Avery's code
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return f"{s} {size_name[i]}"

def get_dir_size(start_path = '.'):
    total_size = 0
    total_files = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_files += 1
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return convert_size(total_size), total_files

In [ ]:
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_pad"))
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_pad/PNG"))
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_pad/JPEG"))
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_mim"))
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_mim/PNG"))
print(get_dir_size("/ssd/jpmokc/analysis/Notebooks/test_imgs_mim/JPEG"))

In [ ]:
#Avery's code
def print_image_shapes(folder_path):
    for dirpath, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(dirpath, filename)
                img = skimage.io.imread(image_path)
                print(f"Image {filename} shape: {img.shape}")
                
def process_image(source_file_path, image_folder, file_format):
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(), xrv.datasets.XRayResizer(224)])
    dest_dir_path = os.path.join(image_folder, file_format.upper())
    os.makedirs(dest_dir_path, exist_ok=True)
    base_filename, _ = os.path.splitext(os.path.basename(source_file_path))
    dest_file_path = os.path.join(dest_dir_path, f"{base_filename}.{file_format.lower()}")

    img = skimage.io.imread(source_file_path)
    img = img / img.max()

    if len(img.shape) == 3:
        if img.shape[2] == 4:
            img = skimage.color.rgba2rgb(img)
        img = skimage.color.rgb2gray(img)
    elif len(img.shape) < 2:
        print("error, dimension lower than 2 for image")

    img = img[None, :, :]

    if transform is not None:
        img = transform(img)

    img = torch.from_numpy(img)

    pil_transform = torchvision.transforms.ToPILImage(mode='L')
    img = pil_transform(img)
    # Save the transformed image as PNG
    img.save(dest_file_path, format='PNG')

def process_images_in_folder(image_folder):
    original_size, original_files = get_dir_size(image_folder)
    print(f"Original size of folder {image_folder}: {original_size}")

    image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.jpg') or filename.endswith('.png')]

    for file_format in ['PNG']:
        with ThreadPoolExecutor() as executor:
            list(executor.map(process_image, image_paths, [image_folder]*len(image_paths), [file_format]*len(image_paths)))

    png_size, png_files = get_dir_size(os.path.join(image_folder, 'PNG'))
    print(f" New PNG folder size: {png_size}")
    # print_image_shapes(os.path.join(image_folder, 'JPG'))
    # print_image_shapes(os.path.join(image_folder, 'PNG'))


In [ ]:
mimic_path = "/ssd/averijordan/datasets/physionet.org/files/mimic-cxr-jpg/2.0.0/files"
mimic_reduced_path = "/ssd/averijordan/datasets/training/data/MIMIC"

pad_path= "/ssd/averijordan/datasets/PADCHEST_SJ/image_zips"
pad_reduced_path = "/ssd/averijordan/datasets/training/data/PadChest"

print(get_dir_size(mimic_path))
print(get_dir_size(mimic_reduced_path))
print(get_dir_size(pad_path))
print(get_dir_size(pad_reduced_path))

In [ ]:
directories = [
    "/ssd2/averijordan/datasets/PadChest/No-Finding/PA"
    "/ssd2/averijordan/datasets/Cardiomegaly/No-Finding/PA"

]

# Process each directory
for directory in directories:
    process_images_in_folder(directory)

In [ ]:

# process_images_in_folder(mimic_dir_eff_pa)
# process_images_in_folder(mimic_dir_eff_ap)
# process_images_in_folder(mimic_dir_norm_pa)
# process_images_in_folder(mimic_dir_norm_ap)
# process_images_in_folder(mimic_dir_pnu_pa)
# process_images_in_folder(mimic_dir_pnu_ap)
# process_images_in_folder(pad_dir_eff_pa)
# process_images_in_folder(pad_dir_eff_ap)
# process_images_in_folder(pad_dir_norm_pa)
# process_images_in_folder(pad_dir_norm_ap)
# process_images_in_folder(pad_dir_pnu_pa)
# process_images_in_folder(pad_dir_pnu_ap)


In [ ]:
process_images_in_folder("/ssd2/pipeline/data/PadChest/no-finding/PA/")

In [ ]:
print_image_shapes("/ssd2/pipeline/data/PadChest/no-finding/PA/")